In [1]:
import requests
from IPython.display import Markdown
from tqdm import tqdm, tqdm_notebook
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import altair as alt
from requests.utils import quote
import os
from datetime import timedelta
from mod import alt_theme

In [2]:
fmt = "{:%Y-%m-%d}"

# Can optionally use number of days to choose dates
n_days = 60
end_date = fmt.format(pd.datetime.today())
start_date = fmt.format(pd.datetime.today() - timedelta(days=n_days))

renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]
bot_names = ["stale", "codecov", "jupyterlab-dev-mode", "henchbot"]

In [3]:
# Parameters
renderer = "kaggle"
start_date = "2019-06-01"
end_date = "2019-07-01"


In [4]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [5]:
# Discourse API key
api = {'Api-Key': os.environ['DISCOURSE_API_KEY'],
       'Api-Username': os.environ['DISCOURSE_API_USERNAME']}

In [6]:
# Discourse
def topics_to_markdown(topics, n_list=10):
    body = []
    for _, topic in topics.iterrows():
        title = topic['fancy_title']
        slug = topic['slug']
        posts_count = topic['posts_count']
        url = f'https://discourse.jupyter.org/t/{slug}'
        body.append(f'* [{title}]({url}) ({posts_count} posts)')
    body = body[:n_list]
    return '\n'.join(body)

def counts_from_activity(activity):
    counts = activity.groupby('category_id').count()['bookmarked'].reset_index()
    counts['parent_category'] = None
    for ii, irow in counts.iterrows():
        if parent_categories[irow['category_id']] is not None:
            counts.loc[ii, 'parent_category'] = parent_categories[irow['category_id']]

    counts['category_id'] = counts['category_id'].map(lambda a: category_mapping[a])
    counts['parent_category'] = counts['parent_category'].map(lambda a: category_mapping[a] if a is not None else 'parent')
    is_parent = counts['parent_category'] == 'parent'
    counts.loc[is_parent, 'parent_category'] = counts.loc[is_parent, 'category_id'] 
    counts['parent/category'] = counts.apply(lambda a: a['parent_category']+'/'+a['category_id'], axis=1)
    counts = counts.sort_values(['parent_category', 'bookmarked'], ascending=False)
    return counts

# Community forum activity

The [Jupyter Community Forum](https://discourse.jupyter.org) is a place for Jovyans across the
community to talk about Jupyter tools in interactive computing and how they fit into their
workflows. It's also a place for developers to share ideas, tools, tips, and help one another.

Below are a few updates from activity in the Discourse. For more detailed information about
the activity on the Community Forum, check out these links:

* [The users page](https://discourse.jupyter.org/u) has information about user activity
* [The top posts page](https://discourse.jupyter.org/top) contains a list of top posts, sorted
  by various metrics.

In [7]:
# Get categories for IDs
url = "https://discourse.jupyter.org/site.json"
resp = requests.get(url, headers=api)
category_mapping = {cat['id']: cat['name'] for cat in resp.json()['categories']}
parent_categories = {cat['id']: cat.get("parent_category_id", None) for cat in resp.json()['categories']}

In [8]:
# Base URL to use
url = "https://discourse.jupyter.org/latest.json"

## Topics with lots of likes

"Likes" are a way for community members to say thanks for a helpful post, show their
support for an idea, or generally to share a little positivity with somebody else.
These are topics that have generated lots of likes in recent history.

In [9]:
params = {"order": "likes", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
liked = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(liked))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (108 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (60 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (38 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (24 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [What&rsquo;s an alternative to the word &ldquo;Fellow&rdquo; or &ldquo;Fellowship&rdquo;?](https://discourse.jupyter.org/t/whats-an-alternative-to-the-word-fellow-or-fellowship) (16 posts)
* [Help us choose an updated &ldquo;Launch Binder&rdquo; badge!](https://discourse.jupyter.org/t/help-us-choose-an-updated-launch-binder-badge) (21 posts)
* [PyCon 2019 and mybinder.org](https://discourse.jupyter.org/t/pycon-2019-and-mybinder-org) (17 posts)
* [The Binder Federation](https://discourse.jupyter.org/t/the-binder-federation) (14 posts)
* [Jupyter community workshop: proposal on diversity, inclusion and accessibility](https://discourse.jupyter.org/t/jupyter-community-workshop-proposal-on-diversity-inclusion-and-accessibility) (20 posts)

## Active topics on the Community Forum

These are topics with lots of activity in recent history.

In [10]:
params = {"order": "posts", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most posts in recent history
posts = pd.DataFrame(resp.json()['topic_list']['topics'])
Markdown(topics_to_markdown(posts))

* [Introduce yourself!](https://discourse.jupyter.org/t/introduce-yourself) (108 posts)
* [{WIP} Re-Design of Jupyter.org Website](https://discourse.jupyter.org/t/wip-re-design-of-jupyter-org-website) (60 posts)
* [Tip: embed custom github content in a Binder link with nbgitpuller](https://discourse.jupyter.org/t/tip-embed-custom-github-content-in-a-binder-link-with-nbgitpuller) (52 posts)
* [JupyterHub, Binder workshop in Oslo, Sept. 5-8](https://discourse.jupyter.org/t/jupyterhub-binder-workshop-in-oslo-sept-5-8) (38 posts)
* [Would a &ldquo;The Littlest Binder&rdquo; be useful?](https://discourse.jupyter.org/t/would-a-the-littlest-binder-be-useful) (31 posts)
* [Jupyter Community Calls](https://discourse.jupyter.org/t/jupyter-community-calls) (29 posts)
* [Plugins of considerations](https://discourse.jupyter.org/t/plugins-of-considerations) (28 posts)
* [A proposal for JupyterHub communications](https://discourse.jupyter.org/t/a-proposal-for-jupyterhub-communications) (26 posts)
* [Binder/BinderHub Workshops &ndash; Help Wanted!](https://discourse.jupyter.org/t/binder-binderhub-workshops-help-wanted) (25 posts)
* [Microsoft Word Integration (Intern Project)](https://discourse.jupyter.org/t/microsoft-word-integration-intern-project) (24 posts)

In [11]:
counts = counts_from_activity(posts)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:            category_id  bookmarked     parent_category  \
  11       accessibility           2      Special Topics   
  10      Admin requests           1       Site Feedback   
  12           nbconvert           1            Notebook   
  0                 Meta           4                Meta   
  1        announcements           1                Meta   
  8           JupyterLab           1          JupyterLab   
  6              discuss           3          JupyterHub   
  2           JupyterHub           2          JupyterHub   
  9   Enterprise Gateway           2  Enterprise Gateway   
  3       Community Chat           3      Community Chat   
  7              discuss           5              Binder   
  4               Binder           4              Binder   
  5            BinderHub           1              Binder   
  
                            parent/category  
  11           Special Topics/accessibility  
  10           Site Feedback/Admin requests  
  12                     Notebook/nbconvert  
  0                               Meta/Meta  
  1                      Meta/announcements  
  8                   JupyterLab/JupyterLab  
  6                      JupyterHub/discuss  
  2                   JupyterHub/JupyterHub  
  9   Enterprise Gateway/Enterprise Gateway  
  3           Community Chat/Community Chat  
  7                          Binder/discuss  
  4                           Binder/Binder  
  5                        Binder/BinderHub  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['accessibility', 'Admin requests', 'nbconvert', 'Meta', 'announcements', 'JupyterLab', 'discuss', 'JupyterHub', 'Enterprise Gateway', 'Community Chat', 'discuss', 'Binder', 'BinderHub']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## Recently-created topics

These are topics that were recently created, sorted by the amount of activity
in each one.

In [12]:
params = {"order": "created", "ascending": "False"}
resp = requests.get(url, headers=api, params=params)

# Sort created by the most posted for recently-created posts
created = pd.DataFrame(resp.json()['topic_list']['topics'])
created = created.sort_values('posts_count', ascending=False)
Markdown(topics_to_markdown(created))

* [User interface tweaks on mybinder.org](https://discourse.jupyter.org/t/user-interface-tweaks-on-mybinder-org) (10 posts)
* [Google OAuthenticator extra security check](https://discourse.jupyter.org/t/google-oauthenticator-extra-security-check) (8 posts)
* [Is there a free (even ad-supported) public JupyterHub available?](https://discourse.jupyter.org/t/is-there-a-free-even-ad-supported-public-jupyterhub-available) (6 posts)
* [Strange `core.ZMQbg!3.xx.xxxx` files appearing in file browser following update](https://discourse.jupyter.org/t/strange-core-zmqbg-3-xx-xxxx-files-appearing-in-file-browser-following-update) (6 posts)
* [Image cannot be rebuilt from a previously working environment.yml file](https://discourse.jupyter.org/t/image-cannot-be-rebuilt-from-a-previously-working-environment-yml-file) (5 posts)
* [JupyterHub LDAP server](https://discourse.jupyter.org/t/jupyterhub-ldap-server) (4 posts)
* [Server becomes &ldquo;stale&rdquo; in a matter of days, even hours](https://discourse.jupyter.org/t/server-becomes-stale-in-a-matter-of-days-even-hours) (4 posts)
* [CSS files won&rsquo;t load on binder](https://discourse.jupyter.org/t/css-files-wont-load-on-binder) (4 posts)
* [How to set config.yaml for multiple OAuth providers](https://discourse.jupyter.org/t/how-to-set-config-yaml-for-multiple-oauth-providers) (4 posts)
* [Custom widgets jupyter problem error syntax](https://discourse.jupyter.org/t/custom-widgets-jupyter-problem-error-syntax) (3 posts)

In [13]:
counts = counts_from_activity(created)
alt.Chart(data=counts, width=700, height=300, title="Activity by category").mark_bar().encode(
    x=alt.X("parent/category", sort=alt.Sort(counts['category_id'].values.tolist())),
    y="bookmarked",
    color="parent_category"
)

Chart({
  data:                          category_id  bookmarked parent_category  \
  8                                Q&A           2             Q&A   
  10                          Notebook           0        Notebook   
  3                               Meta           3            Meta   
  2                               jobs           2            Meta   
  7                         JupyterLab           1      JupyterLab   
  0   Zero to JupyterHub on Kubernetes           2      JupyterHub   
  4                         JupyterHub           1      JupyterHub   
  5                     Community Chat           0  Community Chat   
  6                             Binder           2          Binder   
  9                   mybinder.org ops           1          Binder   
  1                          repo help           0          Binder   
  
                                  parent/category  
  8                                       Q&A/Q&A  
  10                            Notebook/Notebook  
  3                                     Meta/Meta  
  2                                     Meta/jobs  
  7                         JupyterLab/JupyterLab  
  0   JupyterHub/Zero to JupyterHub on Kubernetes  
  4                         JupyterHub/JupyterHub  
  5                 Community Chat/Community Chat  
  6                                 Binder/Binder  
  9                       Binder/mybinder.org ops  
  1                              Binder/repo help  ,
  encoding: FacetedEncoding({
    color: Color({
      field: 'parent_category',
      type: 'nominal'
    }),
    x: X({
      field: 'parent/category',
      sort: Sort(['Q&A', 'Notebook', 'Meta', 'jobs', 'JupyterLab', 'Zero to JupyterHub on Kubernetes', 'JupyterHub', 'Community Chat', 'Binder', 'mybinder.org ops', 'repo help']),
      type: 'nominal'
    }),
    y: Y({
      field: 'bookmarked',
      type: 'quantitative'
    })
  }),
  height: 300,
  mark: 'bar',
  title: 'Activity by category',
  width: 700
})

## User activity in the Community Forum

**Top posters**

These people have posted lots of comments, replies, answers, etc in the community forum.

In [14]:
def plot_user_data(users, column, sort=False):
    plt_data = users.sort_values(column, ascending=False).head(50)
    x = alt.X("username", sort=plt_data['username'].tolist()) if sort is True else 'username'
    ch = alt.Chart(data=plt_data).mark_bar().encode(
        x=x,
        y=column
    )
    return ch

In [15]:
url = "https://discourse.jupyter.org/directory_items.json"
params = {"period": "quarterly", "order": "post_count"}
resp = requests.get(url, headers=api, params=params)

# Topics with the most likes in recent history
users = pd.DataFrame(resp.json()['directory_items'])
users['username'] = users['user'].map(lambda a: a['username'])

In [16]:
plot_user_data(users.head(50), 'post_count')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             134          227             329           12         144   
  1     3             128          195             221           27         124   
  2    24              44           13             250            5          53   
  3   220              36            1              97            1          50   
  4     8              67           12             129            4          50   
  5   116              61           27             124            4          38   
  6    12              32            5              92            5          34   
  7    73              20           18             320            3          29   
  8    99              15           41              90            4          23   
  9    11              14           76             221            1          22   
  10  485              20           59              55            1          21   
  11  234              25            7              12            4          21   
  13   30              22           82              50            2          19   
  12  217               9           27              31            0          19   
  15   77              11           10             154            0          13   
  14   59              15            5              16            1          13   
  16   60               7            6              32            0          11   
  17  222              15            6              16            1          10   
  18  674               3            4              42            3          10   
  19   34              18            3              19            3          10   
  20  566               2            0              12            2           9   
  21  556               8            4              16            1           9   
  22  679              10            1              25            0           9   
  23  432               5           20              50            1           9   
  24    7              18           24              27            0           9   
  27  268              13            5              18            3           8   
  25  510               6            0               4            2           8   
  26  686               3            0              30            3           8   
  29  333               1            2               1            0           7   
  30  771              12            2               3            0           7   
  31  419               2            0              10            4           7   
  32  487               0            0              23            4           7   
  33  662              22           16              16            0           7   
  34  660               5           18              10            1           7   
  35  303               1            7              62            2           7   
  36  590               2            1              21            3           7   
  28   71               4            1              15            1           7   
  41  227               2            8              29            0           6   
  44  372               5            5              12            0           6   
  43    9               7            0              10            0           6   
  42  803               2            0              11            1           6   
  38  640               0            0               6            1           6   
  40  642               1            0              30            5           6   
  39  565               5            3               6            0           6   
  37  195               1            0               8            0           6   
  45  562               0            0               4            0           5   
  46  699               1            0              14            5           5   
  

**Forum users, sorted by likes given**

These are Community Forum members that "liked" other people's posts. We appreciate
anybody taking the time to tell someone else they like what they're shared!

In [17]:
plot_user_data(users.head(50), 'likes_given')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             134          227             329           12         144   
  1     3             128          195             221           27         124   
  13   30              22           82              50            2          19   
  9    11              14           76             221            1          22   
  10  485              20           59              55            1          21   
  8    99              15           41              90            4          23   
  5   116              61           27             124            4          38   
  12  217               9           27              31            0          19   
  24    7              18           24              27            0           9   
  23  432               5           20              50            1           9   
  34  660               5           18              10            1           7   
  7    73              20           18             320            3          29   
  33  662              22           16              16            0           7   
  2    24              44           13             250            5          53   
  4     8              67           12             129            4          50   
  15   77              11           10             154            0          13   
  41  227               2            8              29            0           6   
  47  622               4            8              17            1           5   
  35  303               1            7              62            2           7   
  11  234              25            7              12            4          21   
  17  222              15            6              16            1          10   
  16   60               7            6              32            0          11   
  44  372               5            5              12            0           6   
  6    12              32            5              92            5          34   
  27  268              13            5              18            3           8   
  14   59              15            5              16            1          13   
  21  556               8            4              16            1           9   
  18  674               3            4              42            3          10   
  39  565               5            3               6            0           6   
  49  254               1            3               6            0           5   
  19   34              18            3              19            3          10   
  30  771              12            2               3            0           7   
  29  333               1            2               1            0           7   
  28   71               4            1              15            1           7   
  3   220              36            1              97            1          50   
  36  590               2            1              21            3           7   
  22  679              10            1              25            0           9   
  42  803               2            0              11            1           6   
  48  601               1            0              39            3           5   
  46  699               1            0              14            5           5   
  45  562               0            0               4            0           5   
  43    9               7            0              10            0           6   
  40  642               1            0              30            5           6   
  20  566               2            0              12            2           9   
  31  419               2            0              10            4           7   
  38  640               0            0               6            1           6   
  37  195               1            0               8            0           6   
  

**Forum users, sorted by likes received**

These are folks that posted things other people in the Community Forum liked.

In [18]:
plot_user_data(users.head(50), 'likes_received')

Chart({
  data:      id  likes_received  likes_given  topics_entered  topic_count  post_count  \
  0     6             134          227             329           12         144   
  1     3             128          195             221           27         124   
  4     8              67           12             129            4          50   
  5   116              61           27             124            4          38   
  2    24              44           13             250            5          53   
  3   220              36            1              97            1          50   
  6    12              32            5              92            5          34   
  11  234              25            7              12            4          21   
  33  662              22           16              16            0           7   
  13   30              22           82              50            2          19   
  10  485              20           59              55            1          21   
  7    73              20           18             320            3          29   
  19   34              18            3              19            3          10   
  24    7              18           24              27            0           9   
  8    99              15           41              90            4          23   
  17  222              15            6              16            1          10   
  14   59              15            5              16            1          13   
  9    11              14           76             221            1          22   
  27  268              13            5              18            3           8   
  30  771              12            2               3            0           7   
  15   77              11           10             154            0          13   
  22  679              10            1              25            0           9   
  12  217               9           27              31            0          19   
  21  556               8            4              16            1           9   
  16   60               7            6              32            0          11   
  43    9               7            0              10            0           6   
  25  510               6            0               4            2           8   
  23  432               5           20              50            1           9   
  34  660               5           18              10            1           7   
  44  372               5            5              12            0           6   
  39  565               5            3               6            0           6   
  47  622               4            8              17            1           5   
  28   71               4            1              15            1           7   
  26  686               3            0              30            3           8   
  18  674               3            4              42            3          10   
  42  803               2            0              11            1           6   
  31  419               2            0              10            4           7   
  20  566               2            0              12            2           9   
  36  590               2            1              21            3           7   
  41  227               2            8              29            0           6   
  48  601               1            0              39            3           5   
  46  699               1            0              14            5           5   
  49  254               1            3               6            0           5   
  40  642               1            0              30            5           6   
  37  195               1            0               8            0           6   
  35  303               1            7              62            2           7   
  29  333               1            2               1            0           7   
  

In [19]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>